# Create SAS Autoencoder with SWAT

In [31]:
import tensorflow as tf
import pickle


import swat
swat.options.cas.print_messages = True

 
from sasctl import Session
from sasctl.tasks import register_model, publish_model

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import pickle
from sklearn.metrics import accuracy_score


import os
os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'

conn=swat.CAS("eeclxvm068.exnet.sas.com",5570,'gegrab','Demopw123', caslib='gegrab_s')

# Load into CAS

In [32]:
scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_anomaly_scaler.pickle', 'rb'))

sas =pd.read_csv("Data_orig/machine_anomaly.csv") 

sas =sas[sas.TARGET==0]
 

 
#features = scaler.transform(sas.drop(['MACHINE_ID', 'SENSOR_DATE', 'TARGET'], axis=1))
features = (sas.drop(['MACHINE_ID', 'SENSOR_DATE', 'TARGET'], axis=1))
inputs = list(features)
 

castbl = conn.upload_frame(features, casout=dict(name='anomaly', replace=True))
indata="anomaly"


sas_test =pd.read_csv("Data_orig/machine_anomaly_test_1.csv")
features_test = (sas_test.drop(['MACHINE_ID', 'SENSOR_DATE'], axis=1))

inputs_test = list(features)
 
castbl2 = conn.upload_frame(features_test, casout=dict(name='anomaly_test', replace=True))
 
test_data="anomaly_test"
   
sas_test

NOTE: Cloud Analytic Services made the uploaded file available as table ANOMALY in caslib gegrab_s.
NOTE: The table ANOMALY has been created in caslib gegrab_s from binary data uploaded to Cloud Analytic Services.
NOTE: Cloud Analytic Services made the uploaded file available as table ANOMALY_TEST in caslib gegrab_s.
NOTE: The table ANOMALY_TEST has been created in caslib gegrab_s from binary data uploaded to Cloud Analytic Services.


,MACHINE_ID,SENSOR_DATE,S_0,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32,S_33,S_34,S_35,S_36,S_37,S_38,S_39,S_40,S_41,S_42,S_43,S_44,S_45,S_46,S_47,S_48,S_49,S_50,S_51,S_52,S_53,S_54,S_55,S_56,S_57,S_58,S_59,S_60,S_61,S_62,S_63,S_64,S_65,S_66,S_67,S_68,S_69,S_70,S_71,S_72,S_73,S_74,S_75,S_76,S_77,S_78,S_79,S_80,S_81,S_82,S_83,S_84,S_85,S_86,S_87,S_88,S_89,S_90,S_91,S_92,S_93,S_94,S_95,S_96,S_97,S_98,S_99,S_100,S_101,S_102,S_103,S_104,S_105,S_106,S_107,S_108,S_109,S_110,S_111,S_112,S_113,S_114,S_115,S_116,S_117,S_118,S_119,S_120,S_121,S_122,S_123,S_124,S_125,S_126,S_127,S_128,S_129,S_130,S_131,S_132,S_133,S_134,S_135,S_136,S_137,S_138,S_139
0,77126575,2008-10-14,1.513542,1.703299,0.105858,-1.734316,-2.923845,-3.960274,-3.548093,-2.367497,-1.785614,-1.624966,-0.615504,-0.178877,-0.290969,-0.271594,-0.282566,-0.277685,-0.369716,-0.369993,-0.285284,-0.272462,-0.474015,-0.431840,-0.427661,-0.491028,-0.619054,-0.582643,-0.569886,-0.682628,-0.750917,-0.771658,-0.829894,-0.980337,-1.110777,-1.213429,-1.282330,-1.211127,-1.282350,-1.284423,-1.243019,-1.230755,-1.191890,-1.028628,-0.966939,-0.843533,-0.866797,-0.622154,-0.499871,-0.387841,-0.300492,-0.225148,-0.193542,-0.205148,-0.126496,-0.117967,-0.095051,-0.056533,-0.054032,0.004047,0.017289,0.077435,0.040968,0.121091,0.094078,0.172315,0.252726,0.230414,0.379070,0.470814,0.445199,0.447345,0.554139,0.485419,0.458155,0.582861,0.484168,0.534316,0.578282,0.515104,0.495195,0.573534,0.445042,0.512827,0.543171,0.509198,0.391824,0.454223,0.442906,0.431528,0.438712,0.294446,0.351877,0.286843,0.344575,0.204264,0.292393,0.304304,0.334146,0.284980,0.173300,0.382626,0.594215,0.884430,1.195436,1.248970,1.384233,1.688226,2.111734,2.081296,1.944428,1.893807,1.679214,1.379087,1.077965,0.714508,0.123334,-0.217190,-0.287162,-0.425127,-0.373343,-0.348792,-0.340074,-0.304777,-0.223082,-0.246857,-0.329575,-0.220155,-0.193503,-0.184735,-0.129431,-0.085248,-0.112189,0.210645,0.751278,0.872250,1.536242,2.032530,2.320064,1.469294,0.821087,1.658203
1,77126575,2008-10-15,-0.350679,-1.797723,-3.382608,-4.584815,-4.398598,-3.582011,-2.506601,-1.836639,-1.716801,-0.953810,-0.578857,-0.511596,-0.570148,-0.565118,-0.519191,-0.515032,-0.485705,-0.461247,-0.529872,-0.502089,-0.503942,-0.582585,-0.539986,-0.580031,-0.640903,-0.666257,-0.636401,-0.696428,-0.764627,-0.757323,-0.876005,-0.780707,-0.840967,-0.863972,-0.904849,-0.875071,-0.875306,-0.856820,-0.823906,-0.777796,-0.735709,-0.685669,-0.582038,-0.504691,-0.392833,-0.244328,-0.106917,-0.189614,-0.076632,-0.080898,-0.026653,-0.001704,0.048254,0.043109,0.034277,0.099451,0.093505,0.033757,0.069351,0.049523,0.086327,0.136894,0.158521,0.240017,0.224516,0.316550,0.338923,0.356493,0.429239,0.468134,0.508099,0.625257,0.608551,0.677013,0.693873,0.765720,0.659785,0.762713,0.813886,0.681985,0.751692,0.627915,0.677444,0.652939,0.660896,0.576233,0.582499,0.577665,0.511756,0.571165,0.596325,0.565595,0.575077,0.484155,0.632420,0.654227,0.709666,0.908281,0.926640,1.015585,1.006500,1.077837,1.238522,1.331281,1.174348,0.934693,0.870552,0.760773,0.806148,0.771208,0.732482,0.546654,0.293517,0.173408,0.192682,0.103135,0.198790,0.116375,0.099323,0.091175,0.091558,0.120212,0.058149,0.090873,0.092659,0.106724,0.089518,0.221880,0.477458,0.987681,1.336466,1.266816,1.443469,1.673813,1.550276,1.137756,0.863612,0.702984,0.890389,-0.182883
2,77126575,2008-10-16,-2.652819,-3.706776,-3.977027,-3.767593,-3.193039,-2.351071,-1.261787,-1.118983,-0.684726,-0.283175,-0.342502,-0.290697,-0.309568,-0.287987,-0.224282,-0.133477,-0.157910,-0.164018,-0.249777,-0.181006,-0.024832,-0.102471,-0.080807,-0.078268,-0.136317,-0.084078,-0.142792,-0.101323,-0.123277,-0.101809,-0.132246,-0.197241,-0.206755,-0.046532,-0.059179,-0.177349,-0.180217,-0.030714,-0.056322,-0.119632,-0.054926,-0.005065,0.023270,0.104276,0.129343,0.167114,0.144343,0.055183,0.228910,0.249284,0.228934,0.249158,0.236762,0.180997,0.284768,

In [33]:
# print(castbl.head(111))
# print(castbl2.head())
 

In [34]:
#We use the SAS Deep Learning toolkit to create an empty neural network model that we intend to train as an autoencoder
conn.loadactionset('deepLearn')
conn.deepLearn.buildModel(
    model=dict(name='sas_autoencoder',replace=True),
    type = 'DNN'
)

conn.deepLearn.addLayer(
    model='sas_autoencoder',
    #layer=dict(type='INPUT'),  
    layer=dict(type='INPUT', std='std'),
    replace=True,
    name='data'
)

conn.deepLearn.addLayer(
    model='sas_autoencoder',
    layer=dict(type='FC',n=64),
    replace=True,
    srcLayers='data',
    name='encoding1'
)

conn.deepLearn.addLayer(
    model='sas_autoencoder',
    layer=dict(type='FC',n=32),
    replace=True,
    srcLayers='encoding1',
    name='encoding2'
)


conn.deepLearn.addLayer(
    model='sas_autoencoder',
    layer=dict(type='FC',n=16),
    replace=True,
    srcLayers='encoding2',
    name='bottleneck'
)

conn.deepLearn.addLayer(
    model='sas_autoencoder',
    layer=dict(type='FC',n=32),
    replace=True,
    srcLayers='bottleneck',
    name='decoding1'
)

conn.deepLearn.addLayer(
    model='sas_autoencoder',
    layer=dict(type='FC',n=64),
    replace=True,
    srcLayers='decoding1',
    name='decoding2'
)

 

conn.deepLearn.addLayer(
    model='sas_autoencoder',
    layer=dict(type='OUTPUT'),
    replace=True,
    srcLayers='decoding2',
    name='output'
)

NOTE: Added action set 'deepLearn'.


,casLib,Name,Rows,Columns,casTable
0,gegrab_s,sas_autoencoder,81,5,"CASTable('sas_autoencoder', caslib='gegrab_s')"


In [35]:
#Train the model on the normally functioning radar data (the where clause specifies only the normal radar observations)
conn.deepLearn.dlTrain(
    table = dict(name=indata),
    inputs = inputs,
    modelTable='sas_autoencoder',
    modelWeights=dict(name='sas_autoencoder_trained_weights',replace=True),
#These optimizer settings end up being very important for the functionality of the anomaly detection model.
#Changing these settings can allow you to build a model that better reduces fit error
    optimizer = dict(miniBatchSize=60, maxEpochs=90,  loglevel=1,
                     algorithm=dict(method='adam'))
)




conn.deepLearn.dlExportModel(
    casOut=dict(name='sas_autoencoder_astore', replace=True),
    initWeights='sas_autoencoder_trained_weights',
    modelTable='sas_autoencoder',
    layerImageType="WIDE",
    layers=['output'],
     
)


conn.table.promote('sas_autoencoder_astore', drop=True)
conn.table.promote('sas_autoencoder', drop=True)
conn.table.promote('sas_autoencoder_trained_weights', drop=True)

# conn.table.droptable('sas_autoencoder_astore')
# conn.table.droptable('sas_autoencoder')
# conn.table.droptable('sas_autoencoder_trained_weights')



NOTE:  Synchronous mode is enabled.
NOTE:  The total number of parameters is 23388.
NOTE:  The approximate memory cost is 4.00 MB.
NOTE:  The total number of workers is 5.
NOTE:  The total number of threads on each worker is 8.
NOTE:  The total mini-batch size per thread on each worker is 60.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 2400.
NOTE:  The optimization reached the maximum number of epochs.
NOTE:  The total time is       6.87 (s).
NOTE: 159623 bytes were written to the table "sas_autoencoder_astore" in the caslib "gegrab_s".


ERROR: The target table sas_autoencoder_astore of the promotion already exists. Please specify a different name.
ERROR: The action stopped due to errors.
ERROR: The target table sas_autoencoder of the promotion already exists. Please specify a different name.
ERROR: The action stopped due to errors.
ERROR: The target table sas_autoencoder_trained_weights of the promotion already exists. Please specify a different name.
ERROR: The action stopped due to errors.


+ Elapsed: 0.0199s, user: 0.00445s, sys: 0.00689s, mem: 1.69mb

In [36]:
# conn.table.promote('sas_autoencoder_astore', drop=True)
# conn.table.promote('sas_autoencoder', drop=True)
# conn.table.promote('sas_autoencoder_trained_weights', drop=True)

# conn.table.droptable('sas_autoencoder_astore')
# conn.table.droptable('sas_autoencoder')
# conn.table.droptable('sas_autoencoder_trained_weights')


conn.tableinfo()

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,ANOMALY,2919,140,0,utf-8,2022-06-13T14:39:21-04:00,2022-06-13T14:39:21-04:00,2022-06-13T14:39:23-04:00,UTF8,1.970765e+09,1.970765e+09,1.970765e+09,0,0,0,0,,,0,gegrab,,2022-06-13T14:39:21-04:00,1.970765e+09
1,ANOMALY_TEST,10,140,0,utf-8,2022-06-13T14:39:22-04:00,2022-06-13T14:39:22-04:00,2022-06-13T14:39:22-04:00,UTF8,1.970765e+09,1.970765e+09,1.970765e+09,0,0,0,0,,,0,gegrab,,2022-06-13T14:39:21-04:00,1.970765e+09
2,SAS_AUTOENCODER,81,5,0,utf-8,2022-06-13T14:39:23-04:00,2022-06-13T14:39:23-04:00,2022-06-13T14:39:31-04:00,UTF8,1.970765e+09,1.970765e+09,1.970765e+09,0,0,0,0,,,0,gegrab,,,NaN
3,SAS_AUTOENCODER_TRAINED_WEIGHTS,23388,3,0,utf-8,2022-06-13T14:39:30-04:00,2022-06-13T14:39:30-04:00,2022-06-13T14:39:31-04:00,UTF8,1.970765e+09,1.970765e+09,1.970765e+09,0,0,0,0,,,0,gegrab,,,NaN
4,SAS_AUTOENCODER_ASTORE,1,2,0,utf-8,2022-06-13T14:39:30-04:00,2022-06-13T14:39:30-04:00,2022-06-13T14:39:31-04:00,UTF8,1.970765e+09,1.970765e+09,1.970765e+09,0,0,0,0,,,0,gegrab,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,HMEQ GG_TEST_1_HMEQ GG_2022-06-09_13-21-08_OUTPUT,10,12,0,utf-8,2022-06-09T09:21:13-04:00,2022-06-09T09:21:15-04:00,2022-06-09T11:56:49-04:00,UTF8,1.970400e+09,1.970400e+09,1.970409e+09,1,0,0,0,,,0,gegrab,,,NaN
108,XGBOOST_TEST_1_UNDEFINED_2022_06_10_16_08_39_O...,10,1,0,utf-8,2022-06-10T16:08:54-04:00,2022-06-10T16:09:01-04:00,2022-06-13T08:49:59-04:00,UTF8,1.970511e+09,1.970511e+09,1.970744e+09,1,0,0,0,,,0,gegrab,,,NaN
109,LORRY_TEST,10,14,0,utf-8,2022-06-10T17:11:52-04:00,2022-06-10T17:11:52-04:00,2022-06-10T17:32:24-04:00,UTF8,1.970515e+09,1.970515e+09,1.970516e+09,1,0,0,0,LORRY_TEST.sashdat,gegrab_s,0,gegrab,,2022-06-10T17:11:49-04:00,1.970515e+09
110,TEST_TEST_1_TEST_2022-06-10_21-27-24_OUTPUT,10,18,0,utf-8,2022-06-10T17:27:29-04:00,2022-06-10T17:27:54-04:00,2022-06-10T17:28:22-04:00,UTF8,1.970516e+09,1.970516e+09,1.970516e+09,1,0,0,0,,,0,gegrab,,,NaN


In [37]:
# conn.loadactionset('ASTORE')
# conn.astore.describe(rstore=dict(name='SAS_AUTOENCODER_ASTORE'), epcode=True)

In [38]:
#conn.astore.score(rstore='SAS_AUTOENCODER_ASTORE', ds2code='_LayerAct_4_0_0_112_ +111', table=dict(name=indata))

In [39]:
# print(castbl2.head())

In [40]:
#Score all of the data (both normal and anomalous) using the trained autoencoder
conn.loadactionset('deepLearn')
conn.deepLearn.dlScore(
    table=test_data,
    model='sas_autoencoder',
    initWeights='sas_autoencoder_trained_weights',
    copyVars=inputs_test,
    casout=dict(name='scored_sas_autoencoder', replace=True),
#The following syntax is important for using the results of the autoencoder model and is discussed below
    layerOut=dict(name='sas_layers',replace=True),
    layerImageType="WIDE",
    layers=['output']
)

training= conn.CASTable('sas_layers').to_frame() 


training.head()

training_actuals = training[training.columns[0:140]]
training_predict = training[training.columns[140:]]

reconstruction_errors = tf.keras.losses.msle(training_predict, training_actuals)
threshold = np.mean(reconstruction_errors.numpy()) + np.std(reconstruction_errors.numpy())
print(threshold)


pickle.dump(threshold, open('/sasinside/userdata/gegrab/resources/hmeq/sas_machine_anomaly_threshold.pickle', 'wb'))

NOTE: Added action set 'deepLearn'.
NOTE: Due to data distribution, miniBatchSize has been limited to 6.
0.19424563711368137


In [41]:
data =pd.read_csv('Data_orig/machine_anomaly.csv')

threshold = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/sas_machine_anomaly_threshold.pickle', 'rb'))
threshold

0.19424563711368137

In [42]:


test_target =data.iloc[287:297, 142]

sas_df= conn.CASTable('sas_layers').to_frame() 
 
test_data2 = sas_df[sas_df.columns[0:140]]
predictions_test = sas_df[sas_df.columns[140:]]

errors = tf.keras.losses.msle(predictions_test, test_data2)
 
anomaly_mask = pd.Series(errors) > threshold
preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)

print(accuracy_score(preds, test_target ))

errors, threshold, test_target, preds




 
# predictions_test[0]
 
 

0.6


(<tf.Tensor: shape=(10,), dtype=float64, numpy=
 array([0.09551113, 0.0940026 , 0.16314031, 0.07099431, 0.07093191,
        0.16586167, 0.17657089, 0.23229468, 0.17649551, 0.17506658])>,
 0.19424563711368137,
 287    0.0
 288    0.0
 289    0.0
 290    0.0
 291    0.0
 292    1.0
 293    1.0
 294    1.0
 295    1.0
 296    1.0
 Name: TARGET, dtype: float64,
 0    0.0
 1    0.0
 2    0.0
 3    0.0
 4    0.0
 5    0.0
 6    0.0
 7    1.0
 8    0.0
 9    0.0
 dtype: float64)

In [43]:
# astore_auto = conn.CASTable('SAS_AUTOENCODER_ASTORE')

 
# with Session('eeclxvm067', 'gegrab', 'Demopw123', verify_ssl=False):
#     model = register_model(astore_auto, 'SAS Autoencoder Model', 'SAS Autoencoder Project ' ,force=True)


In [116]:
def execute(MACHINE_ID, SENSOR_DATE, S_0, S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, S_15, S_16, S_17, S_18, S_19, S_20, S_21, S_22, S_23, S_24, S_25, S_26, S_27, S_28, S_29, S_30, S_31, S_32, S_33, S_34, S_35, S_36, S_37, S_38, S_39, S_40, S_41, S_42, S_43, S_44, S_45, S_46, S_47, S_48, S_49, S_50, S_51, S_52, S_53, S_54, S_55, S_56, S_57, S_58, S_59, S_60, S_61, S_62, S_63, S_64, S_65, S_66, S_67, S_68, S_69, S_70, S_71, S_72, S_73, S_74, S_75, S_76, S_77, S_78, S_79, S_80, S_81, S_82, S_83, S_84, S_85, S_86, S_87, S_88, S_89, S_90, S_91, S_92, S_93, S_94, S_95, S_96, S_97, S_98, S_99, S_100, S_101, S_102, S_103, S_104, S_105, S_106, S_107, S_108, S_109, S_110, S_111, S_112, S_113, S_114, S_115, S_116, S_117, S_118, S_119, S_120, S_121, S_122, S_123, S_124, S_125, S_126, S_127, S_128, S_129, S_130, S_131, S_132, S_133, S_134, S_135, S_136, S_137, S_138, S_139 ):
    "Output: ANOMALY_FLAG, errors, threshold, max_anomaly_sensor"
    
    import pickle
    import tensorflow as tf
    import pandas as pd
    import numpy as np
    import swat
    import os
    os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'

    conn=swat.CAS("eeclxvm068.exnet.sas.com",5570,'gegrab','Demopw123', caslib='gegrab_s')
    
    inputArray = pd.DataFrame([[MACHINE_ID, SENSOR_DATE, S_0, S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, S_15, S_16, S_17, S_18, S_19, S_20, S_21, S_22, S_23, S_24, S_25, S_26, S_27, S_28, S_29, S_30, S_31, S_32, S_33, S_34, S_35, S_36, S_37, S_38, S_39, S_40, S_41, S_42, S_43, S_44, S_45, S_46, S_47, S_48, S_49, S_50, S_51, S_52, S_53, S_54, S_55, S_56, S_57, S_58, S_59, S_60, S_61, S_62, S_63, S_64, S_65, S_66, S_67, S_68, S_69, S_70, S_71, S_72, S_73, S_74, S_75, S_76, S_77, S_78, S_79, S_80, S_81, S_82, S_83, S_84, S_85, S_86, S_87, S_88, S_89, S_90, S_91, S_92, S_93, S_94, S_95, S_96, S_97, S_98, S_99, S_100, S_101, S_102, S_103, S_104, S_105, S_106, S_107, S_108, S_109, S_110, S_111, S_112, S_113, S_114, S_115, S_116, S_117, S_118, S_119, S_120, S_121, S_122, S_123, S_124, S_125, S_126, S_127, S_128, S_129, S_130, S_131, S_132, S_133, S_134, S_135, S_136, S_137, S_138, S_139]],
                              columns = ["MACHINE_ID", "SENSOR_DATE", "S_0", "S_1", "S_2", "S_3", "S_4", "S_5", "S_6", "S_7", "S_8", "S_9", "S_10", "S_11", "S_12", "S_13", "S_14", "S_15", "S_16", "S_17", "S_18", "S_19", "S_20", "S_21", "S_22", "S_23", "S_24", "S_25", "S_26", "S_27", "S_28", "S_29", "S_30", "S_31", "S_32", "S_33", "S_34", "S_35", "S_36", "S_37", "S_38", "S_39", "S_40", "S_41", "S_42", "S_43", "S_44", "S_45", "S_46", "S_47", "S_48", "S_49", "S_50", "S_51", "S_52", "S_53", "S_54", "S_55", "S_56", "S_57", "S_58", "S_59", "S_60", "S_61", "S_62", "S_63", "S_64", "S_65", "S_66", "S_67", "S_68", "S_69", "S_70", "S_71", "S_72", "S_73", "S_74", "S_75", "S_76", "S_77", "S_78", "S_79", "S_80", "S_81", "S_82", "S_83", "S_84", "S_85", "S_86", "S_87", "S_88", "S_89", "S_90", "S_91", "S_92", "S_93", "S_94", "S_95", "S_96", "S_97", "S_98", "S_99", "S_100", "S_101", "S_102", "S_103", "S_104", "S_105", "S_106", "S_107", "S_108", "S_109", "S_110", "S_111", "S_112", "S_113", "S_114", "S_115", "S_116", "S_117", "S_118", "S_119", "S_120", "S_121", "S_122", "S_123", "S_124", "S_125", "S_126", "S_127", "S_128", "S_129", "S_130", "S_131", "S_132", "S_133", "S_134", "S_135", "S_136", "S_137", "S_138", "S_139"]
                               )
    inputArray.drop(['MACHINE_ID', 'SENSOR_DATE'], axis=1, inplace=True)
    threshold = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/sas_machine_anomaly_threshold.pickle', 'rb'))
    
    castest = conn.upload_frame(inputArray, casout=dict(name='inputArray', replace=True))
    
     
    
    conn.loadactionset('deepLearn')
    conn.deepLearn.dlScore(
    table=castest,
    model='sas_autoencoder',
    initWeights='sas_autoencoder_trained_weights',
    #pyVars=inputs_test,
    #casout=dict(name='scored_ionosphere_autoencoder', replace=True),                                                                                                        
    layerOut=dict(name='sas_layers',replace=True),
    layerImageType="WIDE",
    layers=['output']
    )

    predictions= conn.CASTable('sas_layers').values
    inputArray = inputArray.values
     
     
    
    errors = tf.keras.losses.msle(predictions, inputArray)
    features= np.subtract(predictions, inputArray)**2
    #print(predictions[0][0], inputArray[0][0], features[0][0])
    
    #print(int(np.where(features ==np.amax(features))[1]))
    sensor_flag= 'S_' + str(int(np.where(features ==np.amax(features))[1])) 
      
    
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)

     
    ANOMALY_FLAG =float(preds)
    errors=float(errors)
    threshold=float(threshold)
    
    if ANOMALY_FLAG == 1:
        max_anomaly_sensor = sensor_flag
    else:
        max_anomaly_sensor= ' '
        
    return (ANOMALY_FLAG, errors, threshold, max_anomaly_sensor)



In [117]:
import pandas as pd
X_test2 = pd.read_csv('Data_orig/machine_anomaly_test_1.csv')

 
for i in range(10):
    print(execute(**X_test2.iloc[i]))
    
#print(X_test2.head(1))

NOTE: Cloud Analytic Services made the uploaded file available as table INPUTARRAY in caslib gegrab_s.
NOTE: The table INPUTARRAY has been created in caslib gegrab_s from binary data uploaded to Cloud Analytic Services.
NOTE: Added action set 'deepLearn'.
NOTE: Due to data distribution, miniBatchSize has been limited to 1.
(0.0, 0.0874259825496929, 0.19424563711368137, ' ')
NOTE: Cloud Analytic Services made the uploaded file available as table INPUTARRAY in caslib gegrab_s.
NOTE: The table INPUTARRAY has been created in caslib gegrab_s from binary data uploaded to Cloud Analytic Services.
NOTE: Added action set 'deepLearn'.
NOTE: Due to data distribution, miniBatchSize has been limited to 1.
(0.0, 0.08788803212923134, 0.19424563711368137, ' ')
NOTE: Cloud Analytic Services made the uploaded file available as table INPUTARRAY in caslib gegrab_s.
NOTE: The table INPUTARRAY has been created in caslib gegrab_s from binary data uploaded to Cloud Analytic Services.
NOTE: Added action set 'd

In [118]:
score_code = """
def execute(MACHINE_ID, SENSOR_DATE, S_0, S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, S_15, S_16, S_17, S_18, S_19, S_20, S_21, S_22, S_23, S_24, S_25, S_26, S_27, S_28, S_29, S_30, S_31, S_32, S_33, S_34, S_35, S_36, S_37, S_38, S_39, S_40, S_41, S_42, S_43, S_44, S_45, S_46, S_47, S_48, S_49, S_50, S_51, S_52, S_53, S_54, S_55, S_56, S_57, S_58, S_59, S_60, S_61, S_62, S_63, S_64, S_65, S_66, S_67, S_68, S_69, S_70, S_71, S_72, S_73, S_74, S_75, S_76, S_77, S_78, S_79, S_80, S_81, S_82, S_83, S_84, S_85, S_86, S_87, S_88, S_89, S_90, S_91, S_92, S_93, S_94, S_95, S_96, S_97, S_98, S_99, S_100, S_101, S_102, S_103, S_104, S_105, S_106, S_107, S_108, S_109, S_110, S_111, S_112, S_113, S_114, S_115, S_116, S_117, S_118, S_119, S_120, S_121, S_122, S_123, S_124, S_125, S_126, S_127, S_128, S_129, S_130, S_131, S_132, S_133, S_134, S_135, S_136, S_137, S_138, S_139 ):
    "Output: ANOMALY_FLAG, errors, threshold, max_anomaly_sensor"
    
    import pickle
    import tensorflow as tf
    import pandas as pd
    import swat
    import os
    os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'

    conn=swat.CAS("eeclxvm068.exnet.sas.com",5570,'gegrab','Demopw123', caslib='gegrab_s')
    
    inputArray = pd.DataFrame([[MACHINE_ID, SENSOR_DATE, S_0, S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, S_15, S_16, S_17, S_18, S_19, S_20, S_21, S_22, S_23, S_24, S_25, S_26, S_27, S_28, S_29, S_30, S_31, S_32, S_33, S_34, S_35, S_36, S_37, S_38, S_39, S_40, S_41, S_42, S_43, S_44, S_45, S_46, S_47, S_48, S_49, S_50, S_51, S_52, S_53, S_54, S_55, S_56, S_57, S_58, S_59, S_60, S_61, S_62, S_63, S_64, S_65, S_66, S_67, S_68, S_69, S_70, S_71, S_72, S_73, S_74, S_75, S_76, S_77, S_78, S_79, S_80, S_81, S_82, S_83, S_84, S_85, S_86, S_87, S_88, S_89, S_90, S_91, S_92, S_93, S_94, S_95, S_96, S_97, S_98, S_99, S_100, S_101, S_102, S_103, S_104, S_105, S_106, S_107, S_108, S_109, S_110, S_111, S_112, S_113, S_114, S_115, S_116, S_117, S_118, S_119, S_120, S_121, S_122, S_123, S_124, S_125, S_126, S_127, S_128, S_129, S_130, S_131, S_132, S_133, S_134, S_135, S_136, S_137, S_138, S_139]],
                              columns = ["MACHINE_ID", "SENSOR_DATE", "S_0", "S_1", "S_2", "S_3", "S_4", "S_5", "S_6", "S_7", "S_8", "S_9", "S_10", "S_11", "S_12", "S_13", "S_14", "S_15", "S_16", "S_17", "S_18", "S_19", "S_20", "S_21", "S_22", "S_23", "S_24", "S_25", "S_26", "S_27", "S_28", "S_29", "S_30", "S_31", "S_32", "S_33", "S_34", "S_35", "S_36", "S_37", "S_38", "S_39", "S_40", "S_41", "S_42", "S_43", "S_44", "S_45", "S_46", "S_47", "S_48", "S_49", "S_50", "S_51", "S_52", "S_53", "S_54", "S_55", "S_56", "S_57", "S_58", "S_59", "S_60", "S_61", "S_62", "S_63", "S_64", "S_65", "S_66", "S_67", "S_68", "S_69", "S_70", "S_71", "S_72", "S_73", "S_74", "S_75", "S_76", "S_77", "S_78", "S_79", "S_80", "S_81", "S_82", "S_83", "S_84", "S_85", "S_86", "S_87", "S_88", "S_89", "S_90", "S_91", "S_92", "S_93", "S_94", "S_95", "S_96", "S_97", "S_98", "S_99", "S_100", "S_101", "S_102", "S_103", "S_104", "S_105", "S_106", "S_107", "S_108", "S_109", "S_110", "S_111", "S_112", "S_113", "S_114", "S_115", "S_116", "S_117", "S_118", "S_119", "S_120", "S_121", "S_122", "S_123", "S_124", "S_125", "S_126", "S_127", "S_128", "S_129", "S_130", "S_131", "S_132", "S_133", "S_134", "S_135", "S_136", "S_137", "S_138", "S_139"]
                               )
    inputArray.drop(['MACHINE_ID', 'SENSOR_DATE'], axis=1, inplace=True)
    threshold = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/sas_machine_anomaly_threshold.pickle', 'rb'))
    
    castest = conn.upload_frame(inputArray, casout=dict(name='inputArray', replace=True))
    
     
    
    conn.loadactionset('deepLearn')
    conn.deepLearn.dlScore(
    table=castest,
    model='sas_autoencoder',
    initWeights='sas_autoencoder_trained_weights',
    #pyVars=inputs_test,
    #casout=dict(name='scored_ionosphere_autoencoder', replace=True),                                                                                                        
    layerOut=dict(name='sas_layers',replace=True),
    layerImageType="WIDE",
    layers=['output']
    )

    predictions= conn.CASTable('sas_layers').values
    inputArray = inputArray.values
     
     
    
    errors = tf.keras.losses.msle(predictions, inputArray)
    features= np.subtract(predictions, inputArray)**2
    #print(predictions[0][0], inputArray[0][0], features[0][0])
    
    #print(int(np.where(features ==np.amax(features))[1]))
    sensor_flag= 'S_' + str(int(np.where(features ==np.amax(features))[1])) 
      
    
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)

     
    ANOMALY_FLAG =float(preds)
    errors=float(errors)
    threshold=float(threshold)
    
    if ANOMALY_FLAG == 1:
        max_anomaly_sensor = sensor_flag
    else:
        max_anomaly_sensor= ' '
        
    return (ANOMALY_FLAG, errors, threshold, max_anomaly_sensor)"""


f = open('Data_orig/Machine_SAS_DLPy_Anomaly_MAX.py',"w+")
f.write(score_code)
f.close()